1) Create Google Cloud Project, Google CLoud STorage, Dataproc Cluster
Blog - https://medium.com/google-cloud/apache-spark-and-jupyter-notebooks-made-easy-with-dataproc-component-gateway-fa91d48d6a5a

Code - https://cloud.google.com/solutions/monte-carlo-methods-with-hadoop-spark

2) Learn spark dataframe
https://docs.databricks.com/getting-started/spark/dataframes.html#apache-spark-dataframes-notebook

Free community Edition to learn spark - community.cloud.databricks.com/login.html

References: https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html

pyspark - https://medium.com/@ravishankar_22148/billions-of-rows-milliseconds-of-time-pyspark-starter-guide-c1f984023bf2

3) Tweet-ids stored manually in Google Storage Bucket
- https://github.com/echen102/COVID-19-TweetIDs

4) Python code to craete a Dateframe from all tweet ids from Google Storage bucket
- Learn dataframe

5) Use twart to hydrate tweets inside mapByPartition
- https://github.com/echen102/COVID-19-TweetIDs/blob/master/hydrate.py
- mapByPartition Example: https://towardsdatascience.com/speeding-up-and-perfecting-your-work-using-parallel-computing-8bc2f0c073f8

6) Develop python code and submit it to Dataproc

Example: https://github.com/GoogleCloudPlatform/cloud-dataproc/tree/master/codelabs/spark-nlp/topic_model.py

https://codelabs.developers.google.com/codelabs/spark-nlp/#7

For example:
gcloud dataproc jobs submit pyspark --cluster ${CLUSTER_NAME}\
    --properties=spark.jars.packages=JohnSnowLabs:spark-nlp:2.0.8 \
    --driver-log-levels root=FATAL \
    topic_model.py \
    -- ${BUCKET_NAME}

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName('Tweet Analysis App') \
  .enableHiveSupport()  \
  .getOrCreate()

In [9]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [10]:
from google.cloud import storage

gcs_client = storage.Client()
bucket = gcs_client.bucket('tweet-ids')

#list(bucket.list_blobs(prefix='2020-01/'))

In [11]:
df2 = spark \
  .read \
  .option ( "inferSchema" , "true" ) \
  .csv ( "gs://tweet-ids/2020-01/*" )

In [12]:
df2.printSchema

<bound method DataFrame.printSchema of DataFrame[_c0: bigint]>

In [13]:
cnt = df2.count()
cnt

25014996

In [14]:
df2.rdd.getNumPartitions()

22

In [15]:
df3 = df2.repartition(200)

In [16]:
df3.rdd.getNumPartitions()

200

In [ ]:
def hydrateTweets():
    #use twart api code
    print(hydrateTweet)

#df3.mapPartitions(shouldHydrateTweets)
#save the output of each partition
#df3.partitionBy().options().save(gs://) 
#parquet 

In [17]:
iterator = df3.mapPartitions(
      (it: Iterator[Row]) =>
        it.toList
          .map(row => {
            println(row)
            #use twart api to get the tweets
            dataframeRow = (row.toString)
            dataframeRow
          })
          .iterator
    )
    .toLocalIterator

SyntaxError: invalid syntax (<ipython-input-17-2907a8fe99ca>, line 2)

In [ ]:
import sys
import getpass

!{sys.executable} -m pip install numpy
sys.path
numpy.__path__

print('This job is running as "{}".'.format(getpass.getuser()))
print(sys.executable, sys.version_info)

In [5]:
#!conda list | grep pyspark
#!conda config --append channels conda-forge
#!conda update -n base conda
#!conda install --yes --prefix {sys.prefix} spark-nlp
#!conda search spark-nlp
#import pyspark
#import imp
#pyspark.__path__
#print("Find ModuleInfo :",imp.find_module('pyspark'))
#pyspark.__version__
#!conda install -c johnsnowlabs spark-nlp=2.3.4
#!conda install -c johnsnowlabs spark-nlp
#!conda remove spark-nlp
!pip install spark-nlp==2.2.1
#!conda list | grep spark-nlp
#import imp
#print(imp.find_module('sparknlp'))
#!conda remove spark-nlp
#!conda install -c johnsnowlabs spark-nlp=2.2.1

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
     |████████████████████████████████| 71kB 1.2MB/s 


Data Analysis

In [7]:
import sparknlp
from sparknlp.annotator import Lemmatizer, Stemmer, Tokenizer, Normalizer
from sparknlp.base import DocumentAssembler, Finisher
# A Spark Session is how we interact with Spark SQL to create Dataframes
from pyspark.sql import SparkSession
# These allow us to create a schema for our data
from pyspark.sql.types import StructField, StructType, StringType, LongType

# Spark Pipelines allow us to sequentially add components such as transformers 
from pyspark.ml import Pipeline

# These are components we will incorporate into our pipeline.
from pyspark.ml.feature import StopWordsRemover, CountVectorizer, IDF

# LDA is our model of choice for topic modeling
from pyspark.ml.clustering import LDA

# Some transformers require the usage of other Spark ML functions. We import them here
from pyspark.sql.functions import col, lit, concat

# This will help catch some PySpark errors
from pyspark.sql.utils import AnalysisException